In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [ ]:
# Load the model from the .h5 file
model = load_model('model_NN_final.h5')

In [ ]:
file_path3 = 'data_parsing/Parsed_Data1.csv'
# Load the CSV file into a DataFrame
data1 = pd.read_csv(file_path3)

In [ ]:
# Apply the function to each row
data1['specific_sequence'] = data1.apply(extract_specific_sequence, axis=1)


data1['1st_pos'] = data1['specific_sequence'].apply(lambda x: x[0])
data1['2nd_pos'] = data1['specific_sequence'].apply(lambda x: x[1])
data1['3rd_pos'] = data1['specific_sequence'].apply(lambda x: x[2])
data1['4th_pos'] = data1['specific_sequence'].apply(lambda x: x[3])
data1['5th_pos'] = data1['specific_sequence'].apply(lambda x: x[4])

In [ ]:
seq_col = [col for col in data1.columns.tolist() if col.startswith('sequence_')]
data1.drop(seq_col, axis=1, inplace=True)
data1.drop('specific_sequence', axis=1, inplace=True)
data1['1st_pos'] = data1['1st_pos'].astype('category')
data1['2nd_pos'] = data1['2nd_pos'].astype('category')
data1['3rd_pos'] = data1['3rd_pos'].astype('category')
data1['4th_pos'] = data1['4th_pos'].astype('category')
data1['5th_pos'] = data1['5th_pos'].astype('category')

In [ ]:
grouped_data1 = data1.groupby("transcript_id")

# Apply the standardization function to the "0 mean" column within each group
data1["0 mean standardized"] = grouped_data1["0 mean"].transform(standardize_column_mean)
data1["0 sd standardized"] = grouped_data1["0 sd"].transform(standardize_column_sd)
data1["0 length standardized"] = grouped_data1["0 length"].transform(standardize_column_mean)

In [ ]:
final_predict_dataset1 = data1.drop(columns=['transcript_id','position','0 length', '0 sd', '0 mean', '0 min', '0 max'])

# Perform one-hot encoding for categorical columns
final_predict_dataset1 = pd.get_dummies(final_predict_dataset1, columns=['1st_pos', '2nd_pos', '3rd_pos', '4th_pos', '5th_pos'])


In [ ]:
transcript1 = data1[['transcript_id','position']]
data1_pred_prob = model.predict(final_predict_dataset1)

In [ ]:
data1_pred_p = pd.DataFrame(pd.DataFrame(data1_pred_prob))
data1_pred_p.columns = ['score']
result_data1 = pd.merge(transcript1, data1_pred_p, left_index=True, right_index=True, how='inner')
agg_functions = {'score': 'max'}

result_data1 = result_data1.groupby(['transcript_id', 'position']).agg(agg_functions).reset_index()

In [ ]:
result_data1.to_csv('prediction_generation/Final_Data1_Result.csv', index=False)